# Tutorial: Causal Inference 07 - SHAP Interpretability

Audience:
- Learners who want to explain *why* uplift scores differ across users.

Prerequisites:
- Notebooks 01 to 06.

Learning goals:
- Build a SHAP-based interpretation workflow.
- Identify which features drive uplift ranking.
- Communicate model behavior in plain language.


## Outline

1. Fit uplift learners and choose one score vector.
2. Train a surrogate model to mimic uplift scores.
3. Use SHAP values to rank feature importance.
4. Turn feature importance into domain explanations.


In [ ]:
from pathlib import Path
import sys

project_root = Path.cwd().resolve()
if not (project_root / "src").exists():
    project_root = project_root.parent

sys.path.insert(0, str(project_root / "src"))
import numpy as np
import pandas as pd

from causal_showcase.data import load_marketing_ab_data, train_test_split_prepared
from causal_showcase.evaluation import qini_auc, qini_curve
from causal_showcase.modeling import fit_meta_learners

data_path = project_root / "data" / "raw" / "marketing_ab.csv"
prepared = load_marketing_ab_data(data_path)
train_data, test_data = train_test_split_prepared(prepared)

learner_results = fit_meta_learners(train_data, test_data)

rows = []
for name, res in learner_results.items():
    curve = qini_curve(test_data.outcome, test_data.treatment, res.uplift_scores)
    rows.append({"model": name, "qini_auc": qini_auc(curve)})

qini_df = pd.DataFrame(rows).sort_values("qini_auc", ascending=False)
best_model = str(qini_df.iloc[0]["model"])
best_scores = learner_results[best_model].uplift_scores

print("Selected model for interpretation:", best_model)
qini_df


## Step 1 - Train a surrogate model for explainability

Why surrogate?
- Some causal learners are harder to explain directly.
- A surrogate model approximates uplift scores and gives stable SHAP values.


In [ ]:
from sklearn.ensemble import RandomForestRegressor

x_test = test_data.X.copy()
surrogate = RandomForestRegressor(n_estimators=300, random_state=42)
surrogate.fit(x_test, best_scores)

surrogate_r2 = surrogate.score(x_test, best_scores)
print(f"Surrogate R^2 on uplift scores: {surrogate_r2:.3f}")


## Step 2 - Compute SHAP values and feature ranking

SHAP (not "SHAPE") tells us how much each feature pushes predictions up or down.


In [ ]:
import shap

explainer = shap.TreeExplainer(surrogate)
shap_values = explainer.shap_values(x_test)
shap_array = np.asarray(shap_values)

importance = np.abs(shap_array).mean(axis=0)
importance_df = pd.DataFrame(
    {
        "feature": x_test.columns,
        "mean_abs_shap": importance,
    }
).sort_values("mean_abs_shap", ascending=False)

importance_df.head(15)


## Step 3 - Save SHAP summary plot

This plot helps you visually explain feature influence patterns.


In [ ]:
import matplotlib.pyplot as plt

out_path = project_root / "artifacts" / "figures" / "notebook_shap_summary.png"
out_path.parent.mkdir(parents=True, exist_ok=True)

shap.summary_plot(shap_array, x_test, show=False)
plt.tight_layout()
plt.savefig(out_path, dpi=160, bbox_inches="tight")
plt.close()

print(f"Saved SHAP summary to {out_path}")


## Interpretation exercise

Pick top 3 features and explain in plain language:
- If feature value increases, does predicted uplift tend to increase or decrease?
- Does this direction make business or domain sense?
- What experiment would you run to validate this interpretation?


In [ ]:
top3 = importance_df.head(3)["feature"].tolist()
print("Top 3 features:", top3)
print("Write your interpretation here for each feature.")
